In [34]:
import os, json, glob
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from IPython.display import display, Markdown
import asyncio
from openai import AsyncAzureOpenAI
from configs.credentials_config import API_KEY, ENDPOINT, MODEL, DEPLOYMENT, EMBEDDINGS_API_KEY, EMBEDDINGS_ENDPOINT, EMBEDDINGS_VERSION, EMBEDDINGS_DEPLOYMENT # Crear archivo credentials_config.py con las credenciales de Azure OpenAI siguiendo el template

import torch
import requests
import numpy as np
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from embeddings_wrapper import LangchainSentenceTransformer, AzureOpenAIEmbedder
from typing import Any
from pydantic import BaseModel, ValidationError, Field



/Users/maxi/miniforge3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
VECTORSTORE_PATH = "datasets/fallos_vectorstore_ada-002"
EMBEDDINGS_OPTION = "azure_openai" # "langchain" or "azure_openai"
EMBEDDINGS_MODEL = "mixedbread-ai/mxbai-embed-large-v1"

In [ ]:
# IDEA DE COMO CARGAR VECTORSTORE: MODIFICAR EL ORIGINAL PARA QUE TENGA METADATOS PARA:
# * fuente: nombre del fallo (ej. "8104")
# * seccion: sección del fallo (ej. "Antecedentes", "Fundamentos de Derecho", etc.)

# 1. Cargar el índice FAISS desde el disco
db = FAISS.load_local(
    VECTORSTORE_PATH,
    embeddings=None,
    allow_dangerous_deserialization=True
)

# 2. Cantidad total de vectores (# chunks) 
print("Total de vectores en FAISS index:", db.index.ntotal)

# 3. Revisar cuántos Document guarda el docstore
docs_dict = db.docstore._dict
print("Total de Document en docstore:", len(docs_dict))

# 4. Inspeccionar los primeros 5 Document para ver texto y metadata
print("\n--- Primeros 5 Documentos ---")
for i, (doc_id, doc) in enumerate(docs_dict.items()):
    if i >= 5:
        break
    print(f"ID: {doc_id}")
    print("Texto (fragmento):", repr(doc.page_content[:100]) + "…")
    print("Metadatos:", doc.metadata)
    print("-" * 40)

# 5. Convertir metadata a DataFrame para análisis completo
records = []
for doc in docs_dict.values():
    # Si metadata estuviera vacío, aparecerán valores por defecto
    records.append({
        "fuente":  doc.metadata.get("fuente", None),
        "seccion": doc.metadata.get("seccion", None),
    })

df = pd.DataFrame(records)

# 5.a. Número de fallos distintos y chunks
print("\n=== Estadísticas generales ===")
summary = pd.DataFrame({
    "Total fallos únicos":  [df["fuente"].nunique(dropna=True)],
    "Total chunks":         [len(df)],
    "Secciones únicas":     [df["seccion"].nunique(dropna=True)],
})
display(summary)

# 5.b. Chunks por 'fuente'
chunks_per_fallo = (
    df.groupby("fuente")
      .size()
      .reset_index(name="n_chunks")
      .sort_values("n_chunks", ascending=False)
)
print("\n=== Chunks por fuente (fallo) ===")
display(chunks_per_fallo)

# 5.c. Chunks por 'seccion'
chunks_per_seccion = (
    df.groupby("seccion")
      .size()
      .reset_index(name="n_chunks")
      .sort_values("n_chunks", ascending=False)
)
print("\n=== Chunks por sección ===")
display(chunks_per_seccion)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Total de vectores en FAISS index: 4918
Total de Document en docstore: 4918

--- Primeros 5 Documentos ---
ID: 6c42f9cd-a447-40c4-b3c8-cc7b383a4a69
Texto (fragmento): '"URBANOTEC S.A. S/ QUIEBRA" - Expte. Nº 8985'…
Metadatos: {}
----------------------------------------
ID: 3d97289c-d85d-4a48-9c57-e9340a4457a5
Texto (fragmento): 'En la ciudad de Paraná, capital de la provincia de Entre Ríos, a los veinticinco días del mes de mar'…
Metadatos: {}
----------------------------------------
ID: 0dc1a4e6-b40c-4b06-b020-1380f459886d
Texto (fragmento): '1.- La sentencia de Camara que viene recurrida La sentencia de la Cámara de Apelaciones Sala Primera'…
Metadatos: {}
----------------------------------------
ID: ea826322-316b-4b29-a862-28370371fe45
Texto (fragmento): 'En consecuencia redujo los honorarios de los letrados Angelini y Bargas a la suma de pesos quiniento'…
Metadatos: {}
----------------------------------------
ID: 853f57c5-2dfb-4d1e-9ea2-bd847ffd569f
Texto (fragmento): 'Para así deci

,Total fallos únicos,Total chunks,Secciones únicas
0,0,4918,0



=== Chunks por fuente (fallo) ===


,fuente,n_chunks



=== Chunks por sección ===


,seccion,n_chunks


In [ ]:
if EMBEDDINGS_OPTION == "langchain":
    embedding_model = LangchainSentenceTransformer(model_name = EMBEDDINGS_MODEL)
    
elif EMBEDDINGS_OPTION == "azure_openai":
    embedding_model = AzureOpenAIEmbedder(
        deployment_name = EMBEDDINGS_DEPLOYMENT,
        endpoint        = EMBEDDINGS_ENDPOINT,
        api_key         = EMBEDDINGS_API_KEY,
        api_version     = EMBEDDINGS_VERSION,
    )


vectorstore = FAISS.load_local(
    "datasets/fallos_vectorstore_ada-002", 
    embedding_model,
    allow_dangerous_deserialization=True
)


def create_context(query):
    results = vectorstore.similarity_search(query, k=5)
    # for i, doc in enumerate(results, 1):
    #     print(f"{i}. {doc.page_content.strip()}\n")
    return results

azure_client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=ENDPOINT,
    api_key=API_KEY
)

async def get_response(query):
    res = await azure_client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": "Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. Tu tarea es analizar el contexto y responder de manera precisa y concisa.",
            },
            {
                "role": "system", 
                "content": f"El contexto es: {create_context(query)}"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        
    )
    return res

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [38]:
response = await get_response("Explicame el fallo que dice: TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS -  Expte. Nº 8142")

In [39]:
respuesta = response.choices[0].message.content
display(Markdown(respuesta))

El fallo correspondiente al expediente "TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS" se refiere a un proceso judicial en el cual Alcides Vicente Terenzano demanda a Marta Olga Landivar y a otro demandado por un cobro de pesos. 

El tribunal considera la solicitud de regulación de honorarios presentada el 7 de diciembre de 2023, relacionada con la actuación profesional en recursos de inaplicabilidad de ley que fueron resueltos en fechas anteriores (25 de junio de 2020 y 10 de abril de 2023). Para este proceso, se toma en cuenta la normativa arancelaria aplicable, citando varios artículos de la ley 7046 que regulan tales situaciones.

En resumen, el fallo se centra en la regulación de honorarios por el trabajo realizado en los mencionados recursos, ajustándose a la legislación pertinente y la liquidación económica aprobada previamente.